In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys

# Add the project's files to the python path
# file_path = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))  # for .py script
file_path = os.path.dirname(os.path.abspath(''))  # for .ipynb notebook
sys.path.append(file_path)
import json
import pandas as pd
from IPython.display import display

# Load JSONL data
file_path = '/home/valerio/git/superpoint_transformer_vschelbi/logs/partition_parametrization_2.0.jsonl'
data = []
with open(file_path, 'r') as f:
    for line in f:
        data.append(json.loads(line))

Flattened data structure to DataFrame


In [ ]:
records = []
for entry in data[1:]:  # skipping the header
    param_set = entry["param_set"]
    for dataset, metrics in entry["performance_metrics"].items():
        if dataset == "mean_metrics":
            continue
        records.append({
            "param_set": param_set,
            "dataset": dataset,
            "|P_0| / |P_1|": metrics["level_ratios"]["|P_0| / |P_1|"],
            "|P_1| / |P_2|": metrics["level_ratios"]["|P_1| / |P_2|"],
            "miou_p1": metrics["semantic_segmentation_miou_p1"],
            "map_p1": metrics["instance_segmentation_map_p1"],
            "pq_p1": metrics["panoptic_segmentation_pq_p1"],
            "miou_p2": metrics["semantic_segmentation_miou_p2"],
            "map_p2": metrics["instance_segmentation_map_p2"],
            "pq_p2": metrics["panoptic_segmentation_pq_p2"]
        })
        
mean_records = []
for entry in data[1:]:
    param_set = entry["param_set"]
    mean_metrics = entry["performance_metrics"]["mean_metrics"]
    mean_records.append({
        "param_set": param_set,
        "dataset": "mean_metrics",
        "|P_0| / |P_1|": mean_metrics["level_ratios"]["|P_0| / |P_1|"],
        "|P_1| / |P_2|": mean_metrics["level_ratios"]["|P_1| / |P_2|"],
        "miou_p1": mean_metrics["semantic_segmentation_miou_p1"],
        "map_p1": mean_metrics["instance_segmentation_map_p1"],
        "pq_p1": mean_metrics["panoptic_segmentation_pq_p1"],
        "miou_p2": mean_metrics["semantic_segmentation_miou_p2"],
        "map_p2": mean_metrics["instance_segmentation_map_p2"],
        "pq_p2": mean_metrics["panoptic_segmentation_pq_p2"]
    })

In [ ]:
df = pd.DataFrame(records)
mean_df = pd.DataFrame(mean_records)

In [ ]:
def filter_performance(df):
    return df[
        (df["|P_0| / |P_1|"].between(30, 50)) &
        (df["|P_1| / |P_2|"].between(3, 10))
        #(df["miou_p1"] > 0.95) &
        #(df["map_p1"] > 0.95) &
        #(df["pq_p1"] > 0.95)
    ]

filtered_df = filter_performance(df)
filtered_mean_df = filter_performance(mean_df)

In [ ]:
top_2_per_dataset = filtered_df.groupby("dataset").apply(lambda x: x.nlargest(2, 'miou_p1')).reset_index(drop=True)

In [ ]:
top_2_per_dataset

In [ ]:
top_10_mean = filtered_mean_df.nlargest(10, 'miou_p1')

In [ ]:
top_10_mean

In [ ]:
df.to_csv("partition_parametrization_2.0.csv", index=True)
mean_df.to_csv("partition_parametrization_2.0_mean.csv", index=True)

In [ ]:
top_2_per_dataset.to_csv("partition_parametrization_2.0_top_2_per_dataset.csv", index=True)
top_10_mean.to_csv("partition_parametrization_2.0_top_10_mean.csv", index=True)